In [30]:
import os
import numpy as np
import cv2  # OpenCV for image processing
from sklearn.model_selection import train_test_split

def load_data(data_dir):
    labels = []
    
    # Enumerate over the folders for real and fake images
    for label, folder in enumerate(['real', 'fake']):
        folder_path = os.path.join(data_dir, folder)
        # Check if the folder exists
        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} does not exist.")
            continue
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
            if image is None:
                print(f"Warning: Unable to load image {img_path}.")
                continue
            image = cv2.resize(image, (28, 28))  # Resize to 28x28
            image = image.astype('float32') / 255.0  # Normalize pixel values
            images.append(image)
            labels.append(label)  # 0 for real, 1 for fake
    
    images = np.array(images)
    labels = np.array(labels)
    
    # Reshape to fit the input shape of the model
    images = images.reshape(-1, 28, 28, 1)  # Add channel dimension
    return images, labels

# Load training and testing data from separate folders
train_data_dir = 'train'  # Path to the training folder
test_data_dir = 'test'     # Path to the testing folder

# Load training data
X_train, y_train = load_data(train_data_dir)
# Load testing data
X_test, y_test = load_data(test_data_dir)

# Optionally split the training data if you want to use a validation set
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}, Labels shape: {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, Labels shape: {y_test.shape}")

Training data shape: (100000, 28, 28, 1), Labels shape: (100000,)
Testing data shape: (20000, 28, 28, 1), Labels shape: (20000,)


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, LeakyReLU, Dropout, Flatten, Dense

def build_discriminator():
    model = Sequential()
    
    # First Conv Block
    model.add(Conv2D(32, 5, input_shape=(28, 28, 1)))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Second Conv Block
    model.add(Conv2D(64, 5))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.2)) 
    model.add(Dropout(0.4))
    
    # Third Conv Block
    model.add(Conv2D(128, 5))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # Flatten and Dense layer
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification
    
    return model

# Create the discriminator model
discriminator = build_discriminator()

In [32]:
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
# Train the model
history = discriminator.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 132s 42ms/step - accuracy: 0.6911 - loss: 0.7910 - val_accuracy: 0.7499 - val_loss: 0.6363
Epoch 2/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 143s 46ms/step - accuracy: 0.8165 - loss: 0.4297 - val_accuracy: 0.7661 - val_loss: 0.6285
Epoch 3/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 144s 46ms/step - accuracy: 0.8398 - loss: 0.3813 - val_accuracy: 0.8740 - val_loss: 0.3003
Epoch 4/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 139s 45ms/step - accuracy: 0.8570 - loss: 0.3379 - val_accuracy: 0.8751 - val_loss: 0.2926
Epoch 5/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 138s 44ms/step - accuracy: 0.8686 - loss: 0.3108 - val_accuracy: 0.8021 - val_loss: 0.4652
Epoch 6/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 147s 47ms/step - accuracy: 0.8761 - loss: 0.2954 - val_accuracy: 0.8800 - val_loss: 0.2889
Epoch 7/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 138s 44ms/step - accuracy: 0.8841 - loss: 0.2798 - val_accuracy: 0.7853 - val_loss: 0.5143
Epoch 8/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 143s 46ms/step - accuracy: 

In [34]:
loss, accuracy = discriminator.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9061 - loss: 0.3036
Test Accuracy: 71.24%
